In [1]:
import warnings
warnings.filterwarnings("ignore")
from src.dataset import load_tokenized_data
from src.model import LyricsMelodyCNNModel
from src.embeddings import extract_embedding_weights
from tensorflow.python.keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
# x, y, tokenizer, songs = load_tokenized_data(with_melody=True, melody_type='CNN', max_samples=100000) 

In [3]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)                    

In [4]:
import joblib
x, y, tokenizer, songs = joblib.load('CNN_x_y_tokenizer_songs.jblib')
songs = songs.reshape(-1, 128,128,1)
embedding_matrix = extract_embedding_weights()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [00:01<00:00, 367.76it/s]


In [5]:
cnn_model = LyricsMelodyCNNModel(tokenizer, embedding_matrix, 
                                 rnn_type='GRU', 
                                 rnn_units=20, 
                                 bidirectional=False,
                                 dropout=0.4,
                                 train_embedding=True, 
                                 n_filters=20,
                                 is_layer_norm=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 20) 200         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 20) 80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)    

In [6]:
cnn_model.train([x,songs], y, epochs=20, batch_size=64, callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Train on 90000 samples, validate on 10000 samples
Epoch 1/20
90000/90000 [==============================] - 65s 721us/step - loss: 7.0288 - _perplexity: 228.5214 - val_loss: 6.5197 - val__perplexity: 217.5331
Epoch 2/20
90000/90000 [==============================] - 61s 673us/step - loss: 6.2463 - _perplexity: 180.1940 - val_loss: 6.2568 - val__perplexity: 239.5901
Epoch 3/20
90000/90000 [==============================] - 61s 675us/step - loss: 6.0499 - _perplexity: 177.1277 - val_loss: 6.1447 - val__perplexity: 278.7493
Epoch 4/20
90000/90000 [==============================] - 61s 674us/step - loss: 5.9573 - _perplexity: 178.7436 - val_loss: 6.1069 - val__perplexity: 351.6559
Epoch 5/20
90000/90000 [==============================] - 61s 675us/step - loss: 5.9085 - _perplexity: 181.9694 - val_loss: 6.1099 - val__perplexity: 439.6752
Epoch 6/20
90000/90000 [==============================] - 61s 673us/step - loss: 5.8748 - _perplexity: 183.

In [7]:
from keras.models import load_model
cnn_model.model.save('cnn_melody_doc2vec_model.h5')  # creates a HDF5 file 'my_model.h5'

In [10]:
# cnn_model.predict(first_word='sweet-talkin', song=songs[0], n_words=50)

In [11]:
from src.dataset import load_tokenized_test

def parse_to_song(text):
    return text.replace('. ', '\n').replace(' \'', '\'').replace(' n\'t', 'n\'t').replace('eos', '')

def test_report():
    test_data = load_tokenized_test(tokenizer, with_melody=True, melody_type='CNN')
    first_words, song_embeddings = zip(*[(song['X'][0], song['melody_embedding'].reshape(128,128,1)) for song in test_data])
    
    
    for i in range(5):
        curr = test_data[i]
        predicted_lyrics = cnn_model.predict(first_word=first_words[i], song=song_embeddings[i], n_words=250)
        lyrics = parse_to_song(predicted_lyrics)
        print(f"song_name: {curr['song_name']}\nartist: {curr['artist']}\nfirst word: {first_words[i]}\nPredicted song:\n{lyrics}\n\n")
        print("===========================================")
              
test_report()

Loading the songs embedding: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.62it/s]


song_name: eternal flame
artist: the bangles
first word: close
Predicted song:
close marshall cruel wilder twilight blmchen audience fro mothers probably inspire by sometimes expose overtime perhaps kim cal favourite sunglasses kills jones lucky prodded booming gave sobrevivir rough stands boca station confessed carving nineteen hotel players weary diesel first slim fussin oldies woo-oo-oo recording new-born hangin vict'ry picture jiggy anyone liberty alleyway levee sinatra streetz a-nothin chains crackle abilene payin still pad beauty jet dancing dazzling daughter warriors quitter facing upsot frio york slap bow fightin verdict fuss sounding mission praising o'er images seem martha 1965 yall hypnotised nought coladas drives divorce dismembered restraining malts older prayed coucher enter ling weekend eating broomstick sleeved warm measure stan pleasin flip smother blinkered try snoop mick against farms insincere ron-ron peru passage united draws famous ey constellations deseo advice p